In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import sqlite3  # Assuming you're using SQLite for the SQL query
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Create the main window
root = tk.Tk()
root.title('Data Visualization with Tkinter')
root.geometry('600x400')

# Sample data for demonstration purposes (replace this with actual database data)
ticker_data = ["AAPL", "GOOG", "AMZN", "TSLA", "MSFT", "NVDA", "META"]

# Create the widgets for the search box, dropdown, and date inputs
search_var = tk.StringVar()

# Search Entry widget
search_label = ttk.Label(root, text="Enter Ticker Symbol:")
search_label.pack(pady=5)
search_entry = ttk.Entry(root, textvariable=search_var, width=20)
search_entry.pack(pady=5)

# Dropdown for financial statement type (initially disabled)
statement_options = ['Income Statement', 'Balance Sheet', 'Cash Flow Statement']
statement_var = tk.StringVar()
statement_dropdown = ttk.Combobox(root, textvariable=statement_var, values=statement_options, state="disabled")
statement_dropdown.pack(pady=5)

# Min and Max date inputs
min_date_label = ttk.Label(root, text="Enter Min Date (YYYY-MM-DD):")
min_date_label.pack(pady=5)
min_date_entry = ttk.Entry(root, width=20)
min_date_entry.pack(pady=5)

max_date_label = ttk.Label(root, text="Enter Max Date (YYYY-MM-DD):")
max_date_label.pack(pady=5)
max_date_entry = ttk.Entry(root, width=20)
max_date_entry.pack(pady=5)

# Function to update the dynamic search box and populate the statement dropdown
def update_search_results(*args):
    query = search_var.get().upper()
    filtered_tickers = [ticker for ticker in ticker_data if query in ticker]
    
    # If filtered tickers are found, update the dropdown with them
    if filtered_tickers:
        statement_dropdown['values'] = filtered_tickers
        # Enable statement dropdown after a valid ticker is found
        statement_dropdown.config(state="normal")
        if query:
            statement_dropdown.current(0)  # Select the first ticker
    else:
        statement_dropdown['values'] = []
        statement_dropdown.config(state="disabled")  # Disable dropdown if no tickers are found

# Bind the search box to dynamically update the dropdown
search_var.trace("w", update_search_results)

# Function to query data and plot the graph
def fetch_data_and_plot():
    ticker = search_var.get().upper()
    statement_type = statement_var.get()
    min_date = min_date_entry.get()
    max_date = max_date_entry.get()
    
    # Validate inputs
    if not ticker or not statement_type or not min_date or not max_date:
        messagebox.showerror("Input Error", "Please fill in all fields before proceeding.")
        return
    
    try:
        # Create a database connection (adjust your DB path for SQLite)
        connection = sqlite3.connect('financial_data.db')  # Adjust for your DB path
        cursor = connection.cursor()

        # Execute SQL query to fetch data (you should modify this based on your actual schema)
        query = f"""
        SELECT date, value FROM financials 
        WHERE ticker = ? AND statement_type = ? AND date BETWEEN ? AND ?
        """
        cursor.execute(query, (ticker, statement_type, min_date, max_date))
        data = cursor.fetchall()
        
        if data:
            # Unzip data into dates and values
            dates, values = zip(*data)
            plot_graph(dates, values)
        else:
            messagebox.showinfo("No Data", f"No data found for {ticker} from {min_date} to {max_date}.")
        
        connection.close()
    except Exception as e:
        messagebox.showerror("Database Error", f"Error fetching data: {str(e)}")

# Function to plot the graph based on the fetched data
def plot_graph(dates, values):
    fig, ax = plt.subplots()
    ax.plot(dates, values)
    ax.set_title('Financial Data Plot')
    ax.set_xlabel('Date')
    ax.set_ylabel('Value')

    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().pack(pady=20)

    # Enable further interaction after data is plotted
    statement_dropdown.config(state="normal")
    min_date_entry.config(state="normal")
    max_date_entry.config(state="normal")

# Button to trigger data fetching and plotting
fetch_button = ttk.Button(root, text="Fetch Data and Plot", command=fetch_data_and_plot)
fetch_button.pack(pady=20)

# Disable interactive elements initially
statement_dropdown.config(state="disabled")
min_date_entry.config(state="disabled")
max_date_entry.config(state="disabled")

# Run the application
root.mainloop()

